# Rule-based Classification
- builds and an fp-tree-like structure to compress data
- mine the rules from the tree by using an algorithm that was based on class squential rule mining

In [50]:
from collections import OrderedDict
from operator import itemgetter

import math
import pandas
import numpy
import os

## -----------------------------------
### Input Data

`transformed_data.csv` file contains the training set and testing set. 

In [51]:
def load_random_sampling_data():
    """
    Reads the data in the csv file and convert it into pandas dataframe.
    This returns the training and testing set.
    """
    file = os.path.abspath('../files/transformed_data.csv')
    data_frame = pandas.read_csv(file)
    training = []
    testing = []
    
    # convert to list
    for column in data_frame:
        list_ = data_frame.loc[:, column].values.tolist()
        for item in list_:
            # For filtering out the Nan values
            if isinstance(item, str):
                string_array = item.split(',')
            
                if column == 'training':
                    training.append(string_array)
                elif column == 'testing':
                    testing.append(string_array)
            
                string_array = []
                
    return training, testing

In [52]:
raw_training, raw_testing = load_random_sampling_data()

In order to create the tree, the inputs should be a dictionary with the itemsets as the dictionary keys and its frequency as the value. The **create_initial_set()** converts the dataset into a python ordered dictionary.

This also eliminates duplicates in the dataset. This returns all the unique patterns in the dataset and the number of time the pattern occured.

In [53]:
def create_initial_set(dataset):
    """the createTree() function doesn't take the input data as lists. 
    It expects a dictionary with the itemsets as the dictionary keys 
    and the frequency as the value. A createInitSet() function does 
    this conversion for you."""

    dictionary = OrderedDict()

    for transaction in dataset:
        pattern = tuple(transaction)
        
        if pattern in dictionary.keys():
            dictionary[pattern] += 1
        else:
            dictionary[pattern] = 1
            
    return dictionary

In [54]:
transaction_set = create_initial_set(raw_training)

In the original implementation of the FP-growth, the second step rearrange the items per transaction according to a list. Let's denote this list or set as **ordered item set**. The **ordered item set** contains the unique items in the database with their corresponding **support count** (the number of times they apper in the database) and is sorted in a desceding order based on the **support count**. But beacause in this case our item set consist of a group of items (the example below will further explain this point), we will tweak the algorithm a little bit. Our **ordered item set** will contain the category names instead.

Example:
`(('saan', 'pr', 'rb', 'nn', 'cc', 'nn', 'abbreviation'), 1)`

In the transaction above, `('saan', 'pr', 'rb', 'nn', 'cc', 'nn', 'abbreviation')` is the item, not the individual member of the pattern which are `saan`, `pr`, `rb` and soon.



In [55]:
def sort_frequent_item_set(item_set):
    """
    Sort the frequent item set in descending order according to their support count.
    
    item_set -- an OrderedDict() that contains the transactions in the database with their support count
    """
    categories = ['abbreviation', 'entity', 'description', 'human', 'location', 'numeric']
    
    ordered_item_set = {}

    # Populate OrderedDict() with the categories as keys
    for category in categories:
        ordered_item_set[category] = 0
    
    for key in item_set.keys():
        index = len(key)
        category = key[index-1]
        ordered_item_set[category] += 1
        
    return OrderedDict(sorted(ordered_item_set.items(), key=itemgetter(1), reverse=True))
        

In [56]:
ordered_item_set = sort_frequent_item_set(transaction_set)

## -----------------------------------
### Tree Structure

In [57]:
class Node:
    def __init__(self, name, count, parent):
        self.name = name
        self.count = count
        self.parent = parent
        self.children = []
        self.node_link = None
        
    # get the child of this node with the name 'name'
    def get_child(self, name):
        for c in self.children:
            if c.name == name:
                return c
        
        return None
    
    #increments the count variable with a given amount    
    def increment(self, num_occur):
        self.count += num_occur
        
    def display(self, node, depth=0):
        children = node.children
        if depth == 0:
            print(node.name)
        else:
            print("+"*depth, node.name)

        depth += 1
        for child in children:
            self.display(child, depth)  # recursive call

In [58]:
f = Node('f', 1, None)
a = Node('a', 2, None)
y = Node('y', 3, None)
f.children.append(a)
f.children.append(y)
f.children[0].name

'a'

Takes the node in the header table with the specified label and add the target node in the end of th chained nodes linked together by the property `node_link` in each node.

In [59]:
def update_header_table(head_node, target_node):
    """Attach the target node to end of the linked list."""
    
    # Do not use recursion to traverse a linked list!
    while (head_node.node_link != None): 
        head_node = head_node.node_link
    
    head_node.node_link = target_node

The total number of **count a node** has is the total number of its appearance in the whole dataset. This is why **node count** in this function is incremented by the **count of the pattern**.

In [60]:
def grow(items, node, header_table, count):
    """
    Adds a branch in the tree if it does not exist using the sequence given in items.
    """

    # if a node already exist in the tree, increments its
    # count by the number of times the pattern occurs
    if node.get_child(items[0]) != None:
        node.get_child(items[0]).increment(count)
    # if the node does not exist, grow the tree
    else:
        new_tree_node = Node(items[0], count, node)
        node.children.append(new_tree_node)

        if items[0] in header_table.keys():
            if header_table[items[0]] == None:
                header_table[items[0]] = new_tree_node
            else:
                update_header_table(header_table[items[0]], node.children[len(node.children)-1])

    if len(items) > 1:
        grow(items[1::], node.get_child(items[0]), header_table, count)

The **header_table** is a dictionary that has the category as the key. The value of each key is a list of links to category nodes (nodes with names that are in category).

The dataset in the form of `OrderedDict([((<q-word>, <pos-tags>, <category>), <count>)])` where **q-word** is the question word, **pos-tags** is the series of part-of-speech tags the sentence has, **category** is the label of the question and the **count** is the number of occurance of the data in the whole dataset.

In [61]:
def build(dataset):
    """Takes the dataset as an arguments and builds the tree.
    This makes two passes through the dataset. 
    The first pass goes through everything in the dataset and counts 
    the frequency of each term. These are stored in the header table."""
    # holds the list of categories and the link to the nodes associated to them
    header_table = {
        "abbreviation": None,
        "description": None,
        "entity": None,
        "human": None,
        "location": None,
        "numeric": None,
    }

    # initialize the tree with the root node
    tree = Node('Root', 1, None)
    
    for transaction, count in dataset.items():
        items = list(transaction)
        grow(items, tree, header_table, count)
    
    return tree, header_table

In [62]:
tree, header_table = build(transaction_set)
start = tree.get_child("saan")
# tree.display(start)

## -----------------------------------
### Debbuging

In [63]:
for key in header_table:
    count = 0
    node = header_table[key]
    while node != None:
        count += 1
        node = node.node_link
    print(key, count)

entity 243
human 235
location 153
description 586
numeric 141
abbreviation 7


## -----------------------------------
### Mining Rules

Given a leaf node (this is the node that contains the category in our tree), this method will gather all the node_links a category has. 

In [64]:
# leaf_node comes from header table
def find_postfix_path(leaf_node):
    """This function iterates through the linked list until it hits 
    the end. For each item it encounters, it calls ascendTreeV2().
    This list is returned and added to the conditional category base 
    dictionary called condPats."""
    
    # list of all the paths with each category
    conditional_patterns = []

    while leaf_node != None:
        postfix_path = []
        ascend_path(leaf_node, postfix_path)
        conditional_patterns.append(
            list(reversed(list(postfix_path[1:])))
        )

        leaf_node = leaf_node.node_link

    return conditional_patterns

This is the method that will asccend the tree-structure and gather all the nodes in the path of the given cateogry node.

In [65]:
def ascend_path(leaf_node, postfix_path):
    """ Ascends from leaf node to root, collecting the names of 
    items it encounters"""

    if leaf_node.parent != None:
        postfix_path.append((leaf_node.name, leaf_node.count,))
        ascend_path(leaf_node.parent, postfix_path)

The key of the **header_table** is the category. The value of each key is a node which is linked to other nodes that contains the said category. This will help us gather all the paths in the tree that belongs to that category.

In FP-growth, **header_table** contains all of the frequent items found in the dataset in order to gather all the paths in which an item *x* occurs. However, in this algorithm we are only interested in gathering all the paths that has the same category that is why the **header_table** keys are categories.

Another thing, in FP-growth, a node is eliminated alone if its count did not reach the **minimum support** count. We did not do that in this algorithm because one of the arguments being proven is that order of the sentence matters. The removal of that single node will destroy the order of the sentence. That is why instead of singly eliminating a node, the whole path in which that node belong will not qualify to be a part of the final conditional pattern bases.

In [66]:
def mine_rules(header_table, minimum_support):
    mined_rules = []
    
    for key in header_table:
        # Conditional Pattern Bases
        cpb = find_postfix_path(header_table[key])

        # Conditional Frequent Pattern tree
        cft = []

        for index in range(0, len(cpb)):
            for item in range(0, len(cpb[index])):
                # Removes the path that did not reach the minimum support
                if cpb[index][item][1] < minimum_support:
                    break
                elif item == len(cpb[index])-1:
                    cft.append(cpb[index])

        print("Key: ", key, len(cft))
        print("Conditional FP-Tree: ", cft)

        value = []

        for category in cft:
            element = tuple(t[0] for t in category)
            value.append(element)

        mined_rules.append([key, value])
        
    return mined_rules

In [67]:
# 80 is the minimum number of occurances of a pattern (?)
rules = mine_rules(header_table, 80)
# rules

Key:  entity 1
Conditional FP-Tree:  [[('paano', 356), ('pr', 151), ('vb', 86)]]
Key:  human 2
Conditional FP-Tree:  [[('paano', 356), ('pr', 151), ('vb', 86)], [('sino', 342), ('dt', 221), ('vb', 116)]]
Key:  location 2
Conditional FP-Tree:  [[('saan', 351), ('pr', 202), ('vb', 124)], [('saan', 351), ('pr', 202)]]
Key:  description 5
Conditional FP-Tree:  [[('paano', 356), ('pr', 151), ('vb', 86)], [('saan', 351), ('pr', 202), ('vb', 124)], [('ano', 321), ('dt', 244), ('nn', 114)], [('paano', 356), ('rb', 190)], [('kailan', 346), ('pr', 188), ('vb', 96)]]
Key:  numeric 2
Conditional FP-Tree:  [[('saan', 351), ('pr', 202), ('vb', 124)], [('kailan', 346), ('pr', 188), ('vb', 96)]]
Key:  abbreviation 1
Conditional FP-Tree:  [[('saan', 351), ('pr', 202), ('vb', 124)]]


## -----------------------------------
### Conflict Resolution Schemes

In [68]:
def is_subset(rule, pattern):
    r = list(rule)
    p = list(pattern)
    
    if len(r) > len(p):
        return False
    else:
        l = p[:len(r)]
        if r == l:
            return True
        else:
            return False

In [69]:
def get_ordered_rule_set(training_data, rules):
    """Rank all of the mined rules according to their accuracy.
    Form of both data: ((pattern), category)
    Returns the oredered rule set as an array of tuples in the
    form (((pattern), category), accuracy).
    """

    antecedent_counts = {} # no. of occurances of the antecedent
    antConseqCounts = {} # no. of antecedent that matches with the consequent
    
    for rule in rules: # populate dictionaries
        for pattern in rule[1]:
            antecedent_counts[pattern] = 0
            antConseqCounts[(pattern, rule[0])] = 0
    
    # Counts the number of antecedent in training data
    for data in training_data: 
        if data[0] in antecedent_counts.keys():
            antecedent_counts[data[0]] += 1
            
#     print "Antecedent Count Dictionary:\n", antecedent_counts

    # Counts the number of antecedent that matches with the consequent in training data
    for data in training_data:
        if data in antConseqCounts.keys():
            antConseqCounts[data] += 1
#     print(antConseqCounts)
            
#     print "Antecendent Consequence:\n", antConseqCounts
    
    orderedRuleSet = []
    for rule in antConseqCounts:
        accuracy = (float(antConseqCounts[rule])/
                    float(antecedent_counts[rule[0]]))*100
        accuracy = int(accuracy)
        orderedRuleSet.append((rule, accuracy,))
        
    orderedRuleSet = sorted(orderedRuleSet, key=lambda rule: rule[1], 
                            reverse=True)
    print("Ordered Rule Set:\n", orderedRuleSet)
    
    return orderedRuleSet

In [70]:
def get_default_class(test_data):
    """Data form: ((pattern), category)"""
    
    question_word = {'alin': 'entity', 'saan':'location', 'ano':'entity', 
                 'kailan':'numeric', 'paano':'description', 
                 'sino':'human', 'bakit':'description'}
    
    for key in question_word.keys():
        if key in test_data[0]:
            return question_word[key]
        
    return None

## -----------------------------------
### Classification and Testing

In [71]:
def classify(testing, rules, decision_list):
    """testing data form: ((pattern), category)
    rules form: [['category', [(pattern1), (pattern2)]]]
    decision_list form: [(((pattern), 'category'), accuracy)]"""
    
    table = []
    
    for data in testing: # populate table
        row = []
        row.append(data[0])
        
        category = []
        for rule in rules: # question classification
            for pattern in rule[1]:
                if data[0] == pattern:
                    category.append(rule[0])
        
        # Conflict Resolution:
        # - if sentence structure did not trigger any category
        if len(category) == 0:
            for i in range(0, len(rules)): # for each category
                for j in range(0, len(rules[i][1])): # for each pattern in that category 
                    # check if any rule is a subset of a sentence
                    if is_subset(rules[i][1][j], data[0]) == True:
                        category.append(rules[i][0])
                        break
                    # if testing data still didn't trigger any rule
                    # assign default class according to its question word
                    elif  (is_subset(rules[i][1][j], data[0])==False) and (i == len(rules)-1) and (j == len(rules[i][1])-1):
                            default = get_default_class(data[0])
                            if default != None:
                                category.append(default)
                                break
            
            # - more than one category was triggered by the subset
            if len(category)>1:
                default = get_default_class(data[0])
                if default != None:
                    category = []
                    category.append(default)

# For debugging purposes:                  
#             if len(category) > 1:
#                 print("category=0: ")
        # - if sentence structure triggered many category
        elif len(category) > 1:
            ranks = []
            # for each category triggered by data
            for i in range(0, len(category)): 
                # combine data pattern with category as tuples
                tmpD = (data[0], category[i],) 
                
                # for each rule in decision_list
                for rule in decision_list:
                    # if combined pattern and category match rule[i]
                    if tmpD == rule[0]: 
                        # get the index
                        ranks.append(decision_list.index(rule))
#                         print ranks
            
            new_category = []
            new_category.append(category[ranks.index(max(ranks))])
            category = new_category
            
# For debugging purpose:                  
#             if len(category):
#                 print "category=1: ", data[0], category
            
        row.append(category)
        table.append(row)
    
    return table

In [72]:
def initialize_testing_set(testing_set):
    final_set = []
    
    for data in testing_set:
        key = tuple(data[:len(data)-2])
        value = data[len(data)-1]
        final_set.append((key,value,))
    
    return final_set

In [73]:
def initialize_training_set(training_set):
    """training set form: ['anong', 'jj', 'pr', 'vb', 'cc', 
    'vb', 'pr', 'cc', 'pr', 'dt', 'vb', 'pr', 'description']
    Returns final_set in the form: ((pattern), category)"""
    
    final_set = []
    
    for data in training_set:
        lst = list(data)
        pattern = tuple(lst[:len(lst)-1])
        category = lst[len(lst)-1]
        final_set.append((pattern, category,))
            
    return final_set

In [74]:
testing_data = initialize_testing_set(raw_testing)
len(testing_data)
training_data = initialize_training_set(raw_training)

612

In [75]:
decision_list = get_ordered_rule_set(training_data, rules)

Ordered Rule Set:
 [((('saan', 'pr'), 'location'), 100), ((('sino', 'dt', 'vb'), 'human'), 100), ((('ano', 'dt', 'nn'), 'description'), 100), ((('paano', 'rb'), 'description'), 100), ((('paano', 'pr', 'vb'), 'description'), 86), ((('kailan', 'pr', 'vb'), 'numeric'), 75), ((('saan', 'pr', 'vb'), 'location'), 73), ((('kailan', 'pr', 'vb'), 'description'), 24), ((('saan', 'pr', 'vb'), 'description'), 20), ((('paano', 'pr', 'vb'), 'human'), 6), ((('paano', 'pr', 'vb'), 'entity'), 6), ((('saan', 'pr', 'vb'), 'abbreviation'), 3), ((('saan', 'pr', 'vb'), 'numeric'), 2)]


In [76]:
prediction_table = classify(testing_data, rules, decision_list)

## -----------------------------------
### Performance Evaluation

In [77]:
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix

In [78]:
def get_comparison_table(testing_data, table):
    performance = []
    
    for i in range(0, len(testing_data)):
        row = []
        row.append(testing_data[i][1])
        row.append(table[i][1])
        performance.append(row)
    
#     print(performance)
    
    correct = 0
    misclassified = 0
    unclassified = 0
    multiple_category = 0
    
    for item in performance:
        if len(item[1]) > 1:
            multiple_category += 1
        else:
            if item[0] == item[1][0]:
                correct += 1
            elif len(item[1]) == 0:
                unclassified += 1
            elif item[0] != item[1][0]:
                misclassified += 1
            
    print("Total number of testing data: {}".format(len(testing_data)))
    print("Correctly classified items: {}".format(correct))
    print("Misclassified items: {}".format(misclassified))
    print("Unclassified items: {}".format(unclassified))
    print("Multiple Classified items: {}".format(multiple_category))
    
    return performance

In [79]:
comparison = get_comparison_table(testing_data, prediction_table)

Total number of testing data: 612
Correctly classified items: 495
Misclassified items: 117
Unclassified items: 0
Multiple Classified items: 0


In [89]:
categories = ['abbreviation', 'entity', 'description', 'human', 
            'location', 'numeric']
y_category = []
y_pred = []

for row in comparison:
    y_category.append(row[0])
    y_pred.append(row[1][0])

precision_recall_fscore_support(y_category, y_pred, labels=numpy.unique(categories), average='macro')

/home/faienix/.virtualenvs/classification/lib/python3.5/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


(0.69117069395711084, 0.72872214307720551, 0.69513440580362929, None)

In [86]:
# calculate f-score for each category
f1_score(y_category,y_pred, labels=numpy.unique(categories), average=None)

/home/faienix/.virtualenvs/classification/lib/python3.5/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


array([ 0.        ,  0.78082192,  0.72857143,  0.83333333,  0.96732026,
        0.86075949])

In [87]:
# calculate f-score of the classifier
f1_score(y_category, y_pred, average='weighted', labels=numpy.unique(categories))

/home/faienix/.virtualenvs/classification/lib/python3.5/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


0.80947834019097276

In [91]:
# row : actual :: column : predicted 
confusion_matrix(y_category, y_pred, labels=numpy.unique(categories))
# I do not understand this

array([[  0,   0,   3,   0,   0,   0],
       [  0, 171,  57,  13,   1,  17],
       [  0,   0, 102,   1,   0,   0],
       [  0,   4,  13,  80,   1,   0],
       [  0,   0,   2,   0,  74,   1],
       [  0,   4,   0,   0,   0,  68]])